<a href="https://colab.research.google.com/github/Hyun-Jun-Lee/Facial_Dection_PJ/blob/master/Age_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
# UTKFACE dataset 나이, 성별 인식
!kaggle datasets download -d jangedoo/utkface-new

In [ ]:
# CKPULS dataset 감정 분석
!kaggle datasets download -d shawon10/ckplus

In [ ]:
!ls

In [ ]:
!unzip utkface-new.zip

In [ ]:
!unzip ckplus.zip

In [ ]:
ls

In [ ]:
# 라이브러리 정리
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.python.keras.callbacks import EarlyStopping
from google.colab import files
from skimage import io
import pathlib
import PIL
from tensorflow.keras import layers

In [ ]:
# 경로 확인
path_utk = '/content/UTKFace'
path_face_cropped = '/content/crop_part1'


path_face_cropped = pathlib.Path(path_face_cropped)
print(path_face_cropped)
print(len(str(path_face_cropped)))

image_count = len(list(path_face_cropped.glob('*.jpg')))
print(image_count)

In [ ]:
faces = list(path_face_cropped.glob('*.jpg'))
print(faces[1])
img1 = PIL.Image.open(str(faces[1]))
img1

In [ ]:
# 이미지 사이즈
batch = 32
img_height = 210
img_width = 210

# Age Detection Model


In [ ]:
# 레이블 설정
age_class=['0-20','21-40','41-60','61-80','81-116']

In [ ]:
# 데이터프레임
df = pd.DataFrame(columns=['images','age','gender'])
for img in faces:
    new_img = str(img)
    img = new_img.split("/")
    #print(img)
    img_name = img[3]
    img = img_name.split("_")
    #print(img)
    #Making 0 and 1 into male and female to make it more interpretable
    if img[1] == '0':
        img[1] = 'male'
    else:
        img[1] = 'female'
    #Making the age label into age ranges instead
    img[0] = int(img[0])
#     if int(img[0]) > 100:
#         img[0] = '101-116'
    if img[0] >80:
        img[0] = '81-116'
    elif img[0] > 60:
        img[0] = '61-80'
    elif img[0] > 40:
        img[0] = '41-60'
    elif img[0] > 20:
        img[0] = '21-40'
    else:
        img[0] = '0-20'    
    df2 = {'images':new_img,'age':img[0],'gender':img[1]}
    df = df.append(df2,ignore_index=True)
df

In [ ]:
# 데이터 나누기

df_data = df.images
y_data = df.age
y2_data = df.gender
X_train, X_test, y_train, y_test = train_test_split(df_data, y_data, test_size=0.2)

d = {'images':X_train,'age':y_train}
df_train = pd.concat(d,axis=1)
df_train.images

In [ ]:
df_data = df_train.images
y_data = df_train.age
y2_data = df.gender
X_train, X_val, y_train, y_val = train_test_split(df_data, y_data, test_size=0.2)

d = {'images':X_train,'age':y_train}
train = pd.concat(d,axis=1)
train

In [ ]:
# 레이블 확인 
train['age'].value_counts()

In [ ]:
d = {'images':X_val,'age':y_val}
val = pd.concat(d,axis=1)
val

In [ ]:
d = {'images':X_test,'age':y_test}
df_test = pd.concat(d,axis=1)
df_test

In [ ]:
# Generator
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale = 1./255)

train_data = train_gen.flow_from_dataframe(dataframe = train, 
                                           x_col = 'images',
                                           y_col = 'age', seed = 42,
                                           batch_size = batch,
                                           shuffle = True, 
                                           class_mode="sparse",
                                           target_size = (img_height,img_width))

test_data = test_gen.flow_from_dataframe(dataframe = df_test,
                                         x_col = 'images',
                                         y_col = 'age',
                                         batch_size = batch,
                                         shuffle = True,
                                         class_mode='sparse',
                                         target_size = (img_height,img_width))

val_data = train_gen.flow_from_dataframe(dataframe = val, 
                                           x_col = 'images',
                                           y_col = 'age', seed = 42,
                                           batch_size = batch,
                                           shuffle = True, 
                                           class_mode="sparse",
                                           target_size = (img_height,img_width))

In [ ]:
for image_batch, labels_batch in train_data:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
for image_batch, labels_batch in val_data:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
# Model

normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
conv_layer_32 = layers.Conv2D(32,(3,3),activation='relu')
conv_layer_64 = layers.Conv2D(64,3,activation='relu')
conv_layer_16 = layers.Conv2D(16,3,activation='relu')
max_pool = layers.MaxPooling2D()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
callback2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# 나이 인식 model

num_classes = 5

model = tf.keras.Sequential([
  normalization_layer,
  conv_layer_32,
  max_pool,
  conv_layer_32,
  max_pool,
  conv_layer_32,
  max_pool,
  #
  conv_layer_32,
  max_pool,
  conv_layer_32,
  max_pool,
  conv_layer_32,
  max_pool,
  #  
  conv_layer_32,
  max_pool,
  conv_layer_32,
  max_pool,
  conv_layer_32,
  max_pool,
    
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs= 20,
    callbacks = callback,
    shuffle=False
)
eff_epochs = len(history.history['loss'])

In [ ]:
model.summary()

In [ ]:
# 성능확인
results_age = model.evaluate(test_data)
print("test loss, test acc:", results_age)

In [ ]:
uploaded = files.upload()


In [ ]:
path = '/content/testing.jpg'
img = tf.keras.preprocessing.image.load_img(
    path, target_size=(img_height, img_width)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

In [ ]:
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

plt.figure(figsize=(5,5))
plt.imshow(img)
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(age_class[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
path2 = '/content/agetest.jpg'
img2 = tf.keras.preprocessing.image.load_img(
    path2, target_size=(img_height, img_width)
)
img_array2 = tf.keras.preprocessing.image.img_to_array(img2)
img_array2 = tf.expand_dims(img_array2, 0)

In [ ]:
predictions2 = model.predict(img_array2)
score2 = tf.nn.softmax(predictions2[0])

plt.figure(figsize=(5,5))
plt.imshow(img2)
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(age_class[np.argmax(score2)], 100 * np.max(score2))
)